In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [2]:
url_dmv = 'https://raw.githubusercontent.com/kansjs/IBM_applied_capstone_share/master/DMV_grid2.csv'
dmv_grid = pd.read_csv(url_dmv)
dmv_grid.head()

,Lat1,Long1,Lat2,Long2
0,39.27,-76.710000,39.187143,-76.833333
1,39.27,-76.833333,39.187143,-76.956667
2,39.27,-76.956667,39.187143,-77.080000
3,39.27,-77.080000,39.187143,-77.203333
4,39.27,-77.203333,39.187143,-77.326667


In [3]:
dmv_grid.shape

(42, 4)

In [4]:
dmv_test = dmv_grid.head(3).copy()
dmv_test

,Lat1,Long1,Lat2,Long2
0,39.27,-76.710000,39.187143,-76.833333
1,39.27,-76.833333,39.187143,-76.956667
2,39.27,-76.956667,39.187143,-77.080000


In [5]:
dmv_test.shape

(3, 4)

In [1]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [7]:
categoryId = '52f2ab2ebcbc57f1066b8b46'
LIMIT = 100

In [8]:
def getNearbyVenues(latitude1, longitude1, latitude2, longitude2):
    venues_list=[]
   
    for Lat1, Long1, Lat2, Long2 in zip(latitude1, longitude1, latitude2, longitude2):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&intent=browse&ne={},{}&sw={},{}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            Lat1, 
            Long1, 
            Lat2,
            Long2,
            LIMIT,
            categoryId)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  'Venue', 
                  'Latitude', 
                  'Longitude',
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
Supermarkets = getNearbyVenues(latitude1 = dmv_grid['Lat1'], 
                               longitude1 = dmv_grid['Long1'], 
                               latitude2 = dmv_grid['Lat2'], 
                               longitude2 = dmv_grid['Long2']
                                  )

In [10]:
Supermarkets.head()

,Venue,Latitude,Longitude,Venue Category
0,Trader Joe's,39.188164,-76.795467,Supermarket
1,Target,39.197493,-76.813360,Big Box Store
2,Giant,39.210065,-76.778580,Supermarket
3,Green Valley Marketplace,39.206965,-76.729124,Supermarket
4,Safeway,39.251080,-76.818721,Supermarket


In [11]:
Supermarkets.shape

(311, 4)

In [12]:
Supermarkets.to_csv('Supermarkets.2.8.20.csv', index=False)